In [3]:
import planetary_computer as pc
import xarray as xr
from xrspatial import hillshade
from datashader.transfer_functions import shade, stack
from datashader.colors import Elevation
from matplotlib import pyplot as plt
import shapely.wkt
from pystac_client import Client
import pandas as pd
import numpy as np

In [4]:
grid_data = pd.read_csv("./grid_metadata.csv")

In [104]:
elevations = []
for g in  grid_data['wkt']:
    P = shapely.wkt.loads(g)
    coords = [P.exterior.coords[2][0], P.exterior.coords[0][1], P.exterior.coords[0][0], P.exterior.coords[2][1]]
    print(coords)
        
    return_data = catalog.search(collections=["nasadem"], bbox=coords)
    
    return_items = [item for item in return_data.get_items()]
    signed_asset = pc.sign(return_items[0].assets["elevation"])
    
    
    # filter to just regions 
    da = (xr.open_rasterio(signed_asset.href).squeeze().drop("band")[:-1, :-1].coarsen({"y": 5, "x": 5}).mean())
    
    x_coords = np.where((da.x>=coords[0]) & (da.x<= coords[2])) 
    y_coords = np.where((da.y>=coords[1]) & (da.y<= coords[3]))
    
    min_x = min(x_coords[0])
    max_x = max(x_coords[0])

    min_y = min(y_coords[0])
    max_y = max(y_coords[0])
    
    subset =  da[min_x:max_x, min_y:max_y]
    
    elevations.append(np.mean(subset[0].to_numpy()))
        
#     plt.imshow(da)    
#     plt.show()

[121.4808486829302, 24.97766123020391, 121.5257644471362, 25.01836939334328]


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: open_rasterio is Deprecated in favor of rioxarray. For information about transitioning, see: https://corteva.github.io/rioxarray/stable/getting_started/getting_started.html
  


[77.25961601995678, 28.54664454217707, 77.30453178416276, 28.58609243100243]


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: open_rasterio is Deprecated in favor of rioxarray. For information about transitioning, see: https://corteva.github.io/rioxarray/stable/getting_started/getting_started.html
  


[-117.9787405899055, 33.79558357488509, -117.9338248256995, 33.83290166381627]


...

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: open_rasterio is Deprecated in favor of rioxarray. For information about transitioning, see: https://corteva.github.io/rioxarray/stable/getting_started/getting_started.html
  


[-117.8889090614936, 34.13085940091023, -117.8439932972876, 34.16803061743935]


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: open_rasterio is Deprecated in favor of rioxarray. For information about transitioning, see: https://corteva.github.io/rioxarray/stable/getting_started/getting_started.html
  


[-117.3499198910219, 33.64614854761477, -117.3050041268159, 33.68353168758084]


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: open_rasterio is Deprecated in favor of rioxarray. For information about transitioning, see: https://corteva.github.io/rioxarray/stable/getting_started/getting_started.html
  


In [124]:
grid_data['elevations'] = elevations

In [126]:
grid_data.to_csv("elevation_data.csv")

In [125]:
grid_data

,grid_id,location,tz,wkt,elevations
0,1X116,Taipei,Asia/Taipei,"POLYGON ((121.5257644471362 24.97766123020391,...",0.000000
1,1Z2W7,Delhi,Asia/Calcutta,"POLYGON ((77.30453178416276 28.54664454217707,...",216.754286
2,3S31A,Los Angeles (SoCAB),Etc/GMT+8,POLYGON ((-117.9338248256995 33.79558357488509...,301.987692
3,6EIL6,Delhi,Asia/Calcutta,"POLYGON ((77.07995296313287 28.54664454217707,...",224.335714
4,7334C,Delhi,Asia/Calcutta,"POLYGON ((77.12486872733885 28.54664454217707,...",216.007143
5,78V83,Delhi,Asia/Calcutta,"POLYGON ((76.94520567051495 28.54664454217707,...",292.071429
6,7F1D1,Delhi,Asia/Calcutta,"POLYGON ((77.12486872733885 28.58609243100243,...",221.182222
7,8KNI6,Delhi,Asia/Calcutta,"POLYGON ((77.30453178416276 28.46770443564941,...",216.497778
8,90BZ1,Taipei,Asia/Taipei,"POLYGON ((121.5706802113421 25.01836939334328,...",0.000000
9,90S79,Delhi,Asia/Calcutta,"POLYGON ((77.21470025575081 28.62552552598286,...",220.450000
